In [7]:
import os
import sys
import shutil
import glob
#from pathlib import Path

import numpy as np
import librosa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

repos_dir = r'/home/akikun/repos'
sys.path.append(repos_dir)
from sak import display as dp
from sak import signal_processing as sp

imgan_dir = os.path.join(repos_dir, 'Intelligibility-MetricGAN')
sys.path.append(imgan_dir)
#import audio_util as au

In [26]:
corpus_dir = r'/home/common/db/audio_corpora/nele'

wav_clean_dir       = os.path.join(corpus_dir, 'hikari_tts')
wav_train_noise_hoya_dir = os.path.join(corpus_dir, 'train_noise', 'hoya')
wav_train_noise_jr_dir   = os.path.join(corpus_dir, 'train_noise', 'JR')

imetricgan_dir = os.path.join(corpus_dir, 'imetricgan')

n_train_test = 479

In [28]:
# normalize the input data into RMS of 0.02.
wav_paths = glob.glob(os.path.join(wav_train_noise_hoya_dir, 'wavs', '*.wav'))
for i, wav_path in enumerate(wav_paths, start=1):
    print('\r' + '{}/{}'.format(i, len(wav_paths)), end="")
    sp.normalize_rms_file(
       wav_path, 
       wav_path.replace('wavs', 'wavs_normalized'))

58/58

In [31]:
# make directories.
if not os.path.exists(imetricgan_dir):
    os.makedirs(imetricgan_dir)

train_dir = os.path.join(imetricgan_dir, 'train')
train_clean_dir = os.path.join(train_dir, 'clean')
train_noise_dir = os.path.join(train_dir, 'noise')
train_enhanced_dir = os.path.join(train_dir, 'enhanced')

test_dir = os.path.join(imetricgan_dir, 'test')
test_clean_dir = os.path.join(test_dir, 'clean')
test_noise_dir = os.path.join(test_dir, 'noise')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(train_clean_dir):
    os.makedirs(train_clean_dir)
if not os.path.exists(train_noise_dir):
    os.makedirs(train_noise_dir) 
if not os.path.exists(train_enhanced_dir):
    os.makedirs(train_enhanced_dir)
    
if not os.path.exists(test_dir):
    os.makedirs(test_dir)
if not os.path.exists(test_clean_dir):
    os.makedirs(test_clean_dir)
if not os.path.exists(test_noise_dir):
    os.makedirs(test_noise_dir) 

# train-test split.
# clean_wavs = glob.glob(os.path.join(corpus_dir, 'hikari_tts', 'wavs_normalized', '*.wav'))
# clean_train, clean_test = train_test_split(clean_wavs, test_size=n_train_test, random_state=1)

# for wav_path in clean_train:
#     shutil.copy(wav_path, os.path.join(train_clean_dir, os.path.basename(wav_path)))
    
# for wav_path in clean_test:
#     shutil.copy(wav_path, os.path.join(test_clean_dir, os.path.basename(wav_path)))

for wav_path in glob.glob(os.path.join(wav_train_noise_hoya_dir, 'wavs_normalized', '*.wav')):
    shutil.copy(wav_path, os.path.join(train_noise_dir, os.path.basename(wav_path)))

for wav_path in glob.glob(os.path.join(wav_train_noise_jr_dir, 'wavs_normalized', '*.wav')):
    shutil.copy(wav_path, os.path.join(test_noise_dir, os.path.basename(wav_path)))

In [23]:
clean_test

['/home/common/db/audio_corpora/nele/hikari_tts/wavs_normalized/hikari-017_004.wav',
 '/home/common/db/audio_corpora/nele/hikari_tts/wavs_normalized/hikari-030_016.wav',
 '/home/common/db/audio_corpora/nele/hikari_tts/wavs_normalized/hikari-055_084.wav',
 '/home/common/db/audio_corpora/nele/hikari_tts/wavs_normalized/hikari-093_150.wav',
 '/home/common/db/audio_corpora/nele/hikari_tts/wavs_normalized/hikari-056_026.wav',
 '/home/common/db/audio_corpora/nele/hikari_tts/wavs_normalized/hikari-012_053.wav',
 '/home/common/db/audio_corpora/nele/hikari_tts/wavs_normalized/hikari-060_019.wav',
 '/home/common/db/audio_corpora/nele/hikari_tts/wavs_normalized/hikari-056_021.wav',
 '/home/common/db/audio_corpora/nele/hikari_tts/wavs_normalized/hikari-063_122.wav',
 '/home/common/db/audio_corpora/nele/hikari_tts/wavs_normalized/hikari-007_002.wav',
 '/home/common/db/audio_corpora/nele/hikari_tts/wavs_normalized/hikari-061_085.wav',
 '/home/common/db/audio_corpora/nele/hikari_tts/wavs_normalized/h